<a href="https://colab.research.google.com/github/PavelStelmakhV/hw309-selection-of-hyperparameters/blob/main/hw309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import layers
from keras import callbacks

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical


# import numpy as np
# import matplotlib.pyplot as plt
# import tf_keras

# from keras import layers
# from keras import models
# from keras import regularizers
# from keras import callbacks
# from keras import initializers

# from keras.layers import BatchNormalization
# from keras.models import load_model


In [10]:
num_classes = 10

In [11]:
%%capture
%%time
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [12]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [13]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [14]:
x_train.shape[1]

28

In [16]:
# Split the data into training and testing sets
# x_train, x_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Standardize the input features
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

# Build a simple neural network
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(units=128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(Dense(units=128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=6)
mc = ModelCheckpoint('best_model.h5', monitor='categorical_accuracy', mode='max', verbose=0, save_best_only=True)

CPU times: user 80.7 ms, sys: 122 µs, total: 80.9 ms
Wall time: 87.3 ms


In [17]:
%%time
# Train the model
history = model.fit(x_train,
                    y_train,
                    epochs=5,
                    batch_size=128,
                    callbacks=[mc, es],
                    validation_split=0.2,
                    verbose=0)

Epoch 1/5
375/375 [==============================] - ETA: 0s - loss: 0.6780 - accuracy: 0.7606

375/375 [==============================] - 9s 16ms/step - loss: 0.6780 - accuracy: 0.7606 - val_loss: 0.4312 - val_accuracy: 0.8432
Epoch 2/5
374/375 [============================>.] - ETA: 0s - loss: 0.4538 - accuracy: 0.8370

375/375 [==============================] - 7s 17ms/step - loss: 0.4538 - accuracy: 0.8370 - val_loss: 0.3891 - val_accuracy: 0.8573
Epoch 3/5
374/375 [============================>.] - ETA: 0s - loss: 0.4056 - accuracy: 0.8525

375/375 [==============================] - 3s 7ms/step - loss: 0.4057 - accuracy: 0.8524 - val_loss: 0.3785 - val_accuracy: 0.8602
Epoch 4/5
368/375 [============================>.] - ETA: 0s - loss: 0.3859 - accuracy: 0.8604

375/375 [==============================] - 3s 7ms/step - loss: 0.3859 - accuracy: 0.8608 - val_loss: 0.3498 - val_accuracy: 0.8717
Epoch 5/5
370/375 [============================>.] - ETA: 0s - loss: 0.3644 - accuracy: 0.8666

375/375 [==============================] - 3s 7ms/step - loss: 0.3641 - accuracy: 0.8668 - val_loss: 0.3434 - val_accuracy: 0.8740
CPU times: user 21.3 s, sys: 1.35 s, total: 22.7 s
Wall time: 24 s


In [18]:
y_pred_prob = model.predict(x_test)
y_pred_prob

313/313 [==============================] - 1s 2ms/step


array([[1.2968937e-07, 1.0568472e-06, 7.9688398e-09, ..., 1.8769170e-03,
        3.1795586e-09, 9.9758923e-01],
       [9.3245484e-07, 1.4268758e-15, 9.9984694e-01, ..., 3.7379133e-21,
        5.3591931e-15, 1.4678600e-19],
       [1.9393817e-14, 9.9999994e-01, 2.6988512e-19, ..., 6.9205884e-24,
        6.7001984e-16, 5.5653568e-21],
       ...,
       [4.1047215e-06, 1.1114966e-11, 3.4091180e-08, ..., 8.1808222e-09,
        9.9998480e-01, 3.5793455e-13],
       [1.9164392e-11, 9.9999994e-01, 2.3914024e-14, ..., 5.7417494e-16,
        1.3842384e-10, 4.0362382e-15],
       [6.2257222e-08, 1.7807452e-08, 2.3269270e-07, ..., 4.2577194e-05,
        5.4548832e-06, 8.6003411e-08]], dtype=float32)

In [19]:
y_pred = np.argmax(y_pred_prob, axis=1).reshape(-1,1)
y_pred

array([[9],
       [2],
       [1],
       ...,
       [8],
       [1],
       [5]])

In [20]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

313/313 [==============================] - 1s 2ms/step - loss: 0.3410 - accuracy: 0.8848
Test Accuracy: 0.8848000168800354


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118

In [ ]:

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# w_init_tanh = initializers.glorot_normal(seed=42)
# w_init_relu = initializers.HeNormal(seed=42)
# # w_init_relu = initializers.HeUniform(seed=24)
# b_init = initializers.Zeros()